<a href="https://colab.research.google.com/github/ASh-Archives/data_science/blob/main/pet_ya_cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-dotenv -q   # установка библиотеки для работы с .env-файлами

In [2]:
from dotenv import load_dotenv  # нужно для обработки .env-файлов
import os                       # нужно для работы с операционной системой
import requests                 # нужно для отправки HTTP-запросов
import json                     # тоже нужено для обработки JSON

In [3]:
from google.colab import drive  # нужно для работы с Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Укажи путь к .env файлу в Google Drive
env_path = '/content/drive/MyDrive/Colab Notebooks/Проекты Мастерской/OKS/YANDEX_CLOUD_KEYS.env'

# Прочитаем файл .env
load_dotenv(dotenv_path=env_path)

# Проверь, что переменные загрузились
api_key = os.getenv("API_KEY_YANDEX")   # API-ключ
folder_id = os.getenv("FOLDER_ID")      # ID каталога

print(f"API Key loaded: {api_key[:5]}...")
print("Folder ID:", folder_id)

API Key loaded: AQVNy...
Folder ID: b1g4p56uunlfkero39u4


In [5]:
def yandex_gpt_generate(
    api_key: str,
    folder_id: str,
    prompt: str,
    model: str = "yandexgpt",
    temperature: float = 0.1,
    max_tokens: int = 12000,
    timeout: int = 45,
) -> str:
    """
    Генерирует ответ с помощью YandexGPT.

    :param api_key: API-ключ Yandex Cloud
    :param folder_id: ID каталога в Yandex Cloud
    :param prompt: Входной промпт
    :param model: Модель (по умолчанию "yandexgpt")
    :param temperature: Температура генерации
    :param max_tokens: Максимальное количество токенов
    :param timeout: Таймаут запроса
    :return: Сгенерированный текст
    :raises RuntimeError: Если запрос не удался
    """
    headers = {
        "Authorization": f"Api-Key {api_key}",
        "Content-Type": "application/json",
    }

    payload = {
        "modelUri": f"gpt://{folder_id}/{model}",
        "messages": [{"role": "user", "text": prompt}],
        "completionOptions": {
            "temperature": temperature,
            "maxTokens": max_tokens,
        },
    }

    try:
        response = requests.post(
            "https://llm.api.cloud.yandex.net/foundationModels/v1/completion",
            headers=headers,
            json=payload,
            timeout=timeout,
        )
        response.raise_for_status()
        data = response.json()
        return data["result"]["alternatives"][0]["message"]["text"]
    except (requests.RequestException, json.JSONDecodeError, KeyError) as e:
        raise RuntimeError(f"YandexGPT request failed: {str(e)}")

In [6]:
# Эталонная форма файла
reference_form = """
 {'filename': '090b01ca-11a2-49b7-95d6-2e287b09b269.txt',
  'uin': '95185f592137673af6e1e3e9f2a987bbdca98a0c8039beb8ff5ddd15eadc06da09da9cad80d94fd0b06adb551992c370',
  'age': '41',
  'gender': 'мужской',
  'graph': {'entities': [{'id': 'p1',
     'type': 'Пациент',
     'age': 41,
     'gender': 'мужской',
     'uin': '95185f592137673af6e1e3e9f2a987bbdca98a0c8039beb8ff5ddd15eadc06da09da9cad80d94fd0b06adb551992c370'},
    {'id': 'd1',
     'type': 'Болезнь',
     'name': 'Острый субэндокардиальный инфаркт миокарда',
     'status': 'активный'},
    {'id': 's1',
     'type': 'Симптом',
     'name': 'Приступы давящих болей за грудиной',
     'intensity': 'умеренная',
     'duration': 'несколько месяцев',
     'onset_date': None},
    {'id': 'l1',
     'type': 'Лабораторный анализ',
     'name': 'Тропониновый тест',
     'value': '0.038',
     'unit': 'нг/мл',
     'date': '2024-07-17',
     'normal_range': None},
    {'id': 'p1',
     'type': 'Процедура',
     'name': 'Коронароангиография с балонной ангиопластикой и стентированием ПНА',
     'date': '2024-05-24',
     'outcome': 'успешная',
     'complications': 'нет'},
    {'id': 'm1',
     'type': 'Лекарство',
     'name': 'Ацетилсалициловая кислота',
     'dosage': '100 мг',
     'frequency': 'ежедневно',
     'route': 'перорально',
     'duration': None},
    {'id': 'f1',
     'type': 'Фактор риска',
     'name': 'Артериальная гипертония',
     'duration': 'несколько лет',
     'severity': 'высокая'},
    {'id': 'a1',
     'type': 'Анатомическая структура',
     'name': 'Передняя межжелудочковая артерия',
     'location': 'сердце'},
    {'id': 'c1',
     'type': 'Осложнение',
     'name': 'ОСН Killip 1',
     'date': '2024-07-17',
     'severity': 'лёгкая'}],
   'relations': [{'from': 'p1', 'relation': 'страдает', 'to': 'd1'},
    {'from': 'p1', 'relation': 'испытывает', 'to': 's1'},
    {'from': 'd1', 'relation': 'подтверждён', 'to': 'l1'},
    {'from': 'p1', 'relation': 'прошёл', 'to': 'p1'},
    {'from': 'p1', 'relation': 'принимает', 'to': 'm1'},
    {'from': 'f1', 'relation': 'усиливает', 'to': 'd1'},
    {'from': 'p1', 'relation': 'перенёс', 'to': 'c1'}],
   'metadata': {'oks_detected': True,
    'oks_type': 'Инфаркт миокарда без подъёма ST',
    'hosp_death': False,
    'source_filename': None,
    'patient_uin': None}}},
"""

In [7]:
# Заметки
note_1 = """
Примеры отношений:
- [Пациент] — страдает → [Болезнь]
- [Пациент] — принимает → [Лекарство]
- [Болезнь] — вызывает → [Симптом]
- [Болезнь] — осложнена → [Осложнение]
- [Болезнь] — повышает_риск → [ОКС]
- [Процедура] — показала → [Находение]
- [Нахождение] — локализовано_в → [Анатомическая структура]
- [Лабораторный анализ] — подтверждает → [Болезнь]
- [Процедура] — выполнена_на → [Анатомическая структура]
- [Пациент] — госпитализирован → [Событие]
- [Пациент] — умер_в_стационаре → [Событие]
- [Фактор риска] — усиливает → [Болезнь]
- [ОКС] — вызван → [Тромбоз]
"""

note_2 = """
Примеры сущностей: - Пациент (age, gender, uin) — УИН пациента указан в начале документа: "УИН пациента: <значение>"
- Болезнь (name, onset_date, severity: лёгкая / средняя / тяжёлая, status: активный / завершённый / хронический)
- Симптом (name, intensity: лёгкая / умеренная / тяжёлая, duration, onset_date)
- Лабораторный анализ (name, value, unit, date, normal_range)
- Процедура (name, date, outcome, complications: нет / есть)
- Лекарство (name, dosage, frequency, route, duration)
- Фактор риска (name, duration, severity) — например: курение, ожирение, гиперхолестеринемия
- Анатомическая структура (name, location) — например: ПНА, ЛЖ, ВТК, аортальный клапан
- Осложнение (name, date, severity)
- Нахождение (description, modality: ЭКГ, ЭхоКГ, КАГ) — например: "элевация ST", "ФВ 45%", "окклюзия ПНА"
- Событие (name, date) — например: госпитализация, выписка, смерть
"""

In [8]:
# Примеры сущностей
entities = """
[
    {
    "id": "p1", "type": "Пациент",
    "id": "o1", "type": "ОКС",  # основной диагноз в фокусе
    "id": "d1", "type": "Болезнь", "name": "ИБС", "note": "предположение: хроническая ишемическая болезнь сердца, часто сопутствует ОКС",
    "id": "s1", "type": "Симптом", "name": "боль в груди",
    "id": "s2", "type": "Симптом", "name": "одышка",
    "id": "l1", "type": "Лабораторный анализ", "name": "тропонин",
    "id": "l2", "type": "Лабораторный анализ", "name": "липидный профиль",
    "id": "pr1", "type": "Процедура", "name": "коронарография",
    "id": "pr2", "type": "Процедура", "name": "стентирование",
    "id": "pr3", "type": "Процедура", "name": "ЭКГ",
    "id": "pr4", "type": "Процедура", "name": "ЭхоКГ",
    "id": "m1", "type": "Лекарство", "name": "Аспирин",
    "id": "m2", "type": "Лекарство", "name": "Статин",
    "id": "m3", "type": "Лекарство", "name": "Клопидогрель",
    "id": "m4", "type": "Лекарство", "name": "Бета-блокатор",
    "id": "m5", "type": "Лекарство", "name": "Ингибитор АПФ",
    "id": "f1", "type": "Фактор риска", "name": "артериальная гипертония",
    "id": "f2", "type": "Фактор риска", "name": "дислипидемия",
    "id": "f3", "type": "Фактор риска", "name": "курение",
    "id": "f4", "type": "Фактор риска", "name": "ожирение",
    "id": "f5", "type": "Фактор риска", "name": "сахарный диабет",
    "id": "a1", "type": "Анатомическая структура", "name": "левый желудочек",
    "id": "a2", "type": "Анатомическая структура", "name": "правая коронарная артерия",
    "id": "a3", "type": "Анатомическая структура", "name": "аортальный клапан",
    "id": "c1", "type": "Осложнение", "name": "повторный инфаркт миокарда",
    "id": "c2", "type": "Осложнение", "name": "сердечная недостаточность",
    "id": "c3", "type": "Осложнение", "name": "инсульт",
    "id": "t1", "type": "Тромбоз",
    "id": "n1", "type": "Нахождение", "name": "элевация ST", "note": "пример находения; типично для ОКС (предположение: типовое)",
    "id": "n2", "type": "Нахождение", "name": "ФВ 45%", "note": "пример снижения фракции выброса, указывает на риск СН (предположение)",
    "id": "e1", "type": "Событие", "name": "госпитализация",
    "id": "e2", "type": "Событие", "name": "выписка",
    "id": "e3", "type": "Событие", "name": "смерть"
    }
]
"""

In [9]:
relations = """
[
    {
    # базовые связи пациента и диагноза
    "from": "p1", "relation": "страдает", "to": "o1",
    "from": "p1", "relation": "страдает", "to": "d1", "note": "предположение: хроническая ИБС у пациента, часто сопутствует ОКС",

    # факторы риска
    "from": "p1", "relation": "имеет_фактор_риска", "to": "f1",
    "from": "p1", "relation": "имеет_фактор_риска", "to": "f2",
    "from": "p1", "relation": "имеет_фактор_риска", "to": "f3",
    "from": "p1", "relation": "имеет_фактор_риска", "to": "f4",
    "from": "p1", "relation": "имеет_фактор_риска", "to": "f5",

    # назначенное лечение (вторичная профилактика) — отмечено как типичные схемы (общее предположение)
    "from": "p1", "relation": "назначено_лечение", "to": "m1", "note": "типично: антиагрегант (аспирин)",
    "from": "p1", "relation": "назначено_лечение", "to": "m3", "note": "типично: второй антиагрегант (клопидогрель) при ДАТ/стентировании, предположение",
    "from": "p1", "relation": "назначено_лечение", "to": "m2", "note": "типично: статины для вторичной профилактики",
    "from": "p1", "relation": "назначено_лечение", "to": "m4",
    "from": "p1", "relation": "назначено_лечение", "to": "m5",

    # диагностика / процедуры / лаборатория
    "from": "p1", "relation": "проведена_диагностика", "to": "pr1",
    "from": "p1", "relation": "проведена_диагностика", "to": "pr2",
    "from": "p1", "relation": "проведена_диагностика", "to": "pr3",
    "from": "p1", "relation": "проведена_диагностика", "to": "pr4",
    "from": "p1", "relation": "проведена_диагностика", "to": "l1",
    "from": "p1", "relation": "проведена_диагностика", "to": "l2",

    # причинно-следственные связи и находения
    "from": "o1", "relation": "вызывает", "to": "s1",
    "from": "o1", "relation": "вызывает", "to": "s2",
    "from": "o1", "relation": "осложнена", "to": "c1",
    "from": "o1", "relation": "осложнена", "to": "c2",
    "from": "o1", "relation": "вызван", "to": "t1",  # из примера: ОКС может быть вызван тромбозом

    # анализы и подтверждение
    "from": "l1", "relation": "подтверждает", "to": "o1", "note": "тропонин как маркер некроза миокарда",
    "from": "l2", "relation": "повышает_риск", "to": "o1", "note": "дислипидемия как лабораторно выявленное состояние, повышающее риск (логическое соответствие)",

    # процедуры — выполнено на анатомических структурах (часто предполагается)
    "from": "pr1", "relation": "выполнена_на", "to": "a2", "note": "коронарография выполняется на коронарных артериях; конкретная локализация предполагается",
    "from": "pr2", "relation": "выполнена_на", "to": "a2", "note": "стентирование аудиториально связано с поражённой коронарной артерией",
    "from": "pr4", "relation": "выполнена_на", "to": "a1",

    # нахождения — локализовано в
    "from": "n1", "relation": "локализовано_в", "to": "a2", "note": "элевция ST обычно соотносится с зоной коронарной ишемии; привязка — предположение",
    "from": "n2", "relation": "локализовано_в", "to": "a1", "note": "ФВ относится к левому желудочку (предположение)",

    # связи между болезнями/факторами риска/осложнениями
    "from": "d1", "relation": "повышает_риск", "to": "c1", "note": "ИБС повышает риск повторного инфаркта (клиническое предположение)",
    "from": "f1", "relation": "усиливает", "to": "d1",
    "from": "f2", "relation": "усиливает", "to": "d1",
    "from": "p1", "relation": "подвержен_риску", "to": "c1", "note": "на основе наличия ОКС и факторов риска: высокий риск повторных осложнений",
    "from": "p1", "relation": "подвержен_риску", "to": "c2", "note": "риск СН после ОКС — клиническое предположение",

    # события
    "from": "p1", "relation": "госпитализирован", "to": "e1", "note": "госпитализация",
    "from": "p1", "relation": "выписан", "to": "e2", "note": "выписка",
    "from": "p1", "relation": "умер_в_стационаре", "to": "e3", "note": "смерть"
    }
]
"""

In [10]:
# Инструкция для формирования JSON-структуры
reference_instructions = """
1. Извлеките из текста следующие данные:
   - УИН пациента
   - Возраст пациента
   - Пол пациента
   - Диагноз
   - Симптомы
   - Процедуры и манипуляции
   - Результаты лабораторных анализов
   - Осложнения
   - Назначенные лекарства
   - Факторы риска

2. Структурируйте извлечённые данные в формате JSON, используя следующие ключи:
   - `filename`
   - `uin`
   - `age`
   - `gender`
   - `graph`:
     - `entities`:
       - `id`
       - `type` (Пациент, Болезнь, Симптом, Лабораторный анализ, Процедура, Лекарство, Фактор риска, Анатомическая структура, Осложнение)
       - `name`
       - `value` (для лабораторных анализов)
       - `unit` (для лабораторных анализов)
       - `date`
       - `normal_range` (для лабораторных анализов)
       - `dosage` (для лекарств)
       - `frequency` (для лекарств)
       - `route` (для лекарств)
       - `duration` (для лекарств)
       - `location` (для анатомических структур)
       - `severity` (для осложнений)
     - `relations`:
       - `from`
       - `relation`
       - `to`
   - `metadata`:
     - `oks_detected`
     - `oks_type`
     - `hosp_death`
     - `source_filename`
     - `patient_uin`
"""

In [17]:
# Загрузка текста - эпикриз
with open("/content/drive/MyDrive/Colab Notebooks/Проекты Мастерской/OKS/epicrises/090b01ca-11a2-49b7-95d6-2e287b09b269.txt", "r", encoding="utf-8") as f:
    file_content = f.read()

In [18]:
response = yandex_gpt_generate(
    api_key=os.getenv("API_KEY_YANDEX"),
    folder_id=os.getenv("FOLDER_ID"),
    prompt=f"Проанализируй текст №1\n\nТекст №1 - эпикриз:\n\n{file_content}\n\n"
           f"Текст №1 нужно привести к образцовой JSON-структуре как текст №2:\n\nТекст №2:\n{reference_form}\n\n"
           f"В тексте №1 требуется правильно определить id по типу сущности, пример:\n\n{entities}\n\n"
           f"Также в тексте №1 при составлении блока relations требуется точно определить отношение между сущностями, например как в:\n\n{relations}\n\n"
           f"В ответ пришли промт который точно задаст форму по формированию JSON-структуры, форма должна влкючать инструкцию в виде коментариев.\n\n",
)
print(response)

```
Преобразуйте текст №1 в JSON-структуру, следуя примеру текста №2. 

Инструкция:
1. Определите все сущности в тексте №1 и присвойте им уникальные идентификаторы (id) в формате "тип_сущности + порядковый номер". Например, для пациента это будет "p1", для болезни — "d1", для симптома — "s1" и так далее.
2. Для каждой сущности укажите её тип и основные характеристики. Например, для пациента укажите возраст, пол, UIN; для болезни — название, статус; для симптома — название, интенсивность, продолжительность и т. д.
3. Определите отношения между сущностями и укажите их в блоке "relations". Например, "пациент страдает болезнью", "пациент испытывает симптом", "болезнь подтверждена лабораторным анализом" и т. п.
4. Включите в JSON-структуру блок "metadata" с дополнительной информацией о пациенте и его состоянии.

Пример структуры:
```
{
  "filename": "название_файла",
  "uin": "уникальный_идентификатор_пациента",
  "age": "возраст_пациента",
  "gender": "пол_пациента",
  "graph": {
    "enti

**ЭСПЕРЕМЕНТЫ НИЖЕ**

In [14]:
# Вызов функции
response = yandex_gpt_generate(
    api_key=os.getenv("API_KEY_YANDEX"),
    folder_id=os.getenv("FOLDER_ID"),
    prompt=f"Проанализируй текст №1\n\nТекст №1\n\n{file_content}\n\n"
            f"Текст №1 нужно привести к образцовой JSON-структуре по данной инструкции:\n\n{reference_instructions}\n\n"
            f"Для точного определения отношений сущностей в блоке entities и relations используй пример:\n\nentities{entities}\n\nrelations{relations}"
            f"\n\nВ ответ пришли готовую JSON-структуру"
)
print(response)

```
{
  "filename": "history_40382",
  "uin": "722c762d0de3dff5b640147a58296a79239527f1364f535efb307d85321a38f409da9cad80d94fd0b06adb551992c370",
  "age": 59,
  "gender": "мужской",
  "graph": {
    "entities": [
      {
        "id": "p1",
        "type": "Пациент",
        "name": "Пациент"
      },
      {
        "id": "o1",
        "type": "ОКС",
        "name": "Основной диагноз"
      },
      {
        "id": "d1",
        "type": "Болезнь",
        "name": "ИБС: Острый коронарный синдром без подъема СТ"
      },
      {
        "id": "s1",
        "type": "Симптом",
        "name": "слабость"
      },
      {
        "id": "s2",
        "type": "Симптом",
        "name": "одышка"
      },
      {
        "id": "l1",
        "type": "Лабораторный анализ",
        "name": "Общий анализ крови",
        "value": "4,58 10**12/л",
        "unit": "10**12/л",
        "date": "22.09.2024",
        "normal_range": "4,600 - 6,200"
      },
      {
        "id": "l2",
        "type": "Лаб

In [ ]:
# Вызов функции
response = yandex_gpt_generate(
    api_key=os.getenv("API_KEY_YANDEX"),
    folder_id=os.getenv("FOLDER_ID"),
    prompt="Привет! Как дела?",
)
print(response)